In [ ]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('{:.2f}%'.format(sparsity))

In [66]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
 
    """
    Args:
        sparse_data (csr_matrix): 유저와 아이템 행렬 (training set)
 
        alpha_val (int): 신뢰도 값 조정에 필요한 값.
 
        iterations (int): 반복 횟수
 
        lambda_val (float): 정규화에 필요한 값. 정규화 값이 증가 할 수록 편향도 증가 Default is 0.1.
 
        features (int): 계산할 latent features의 값
    
    Returns:     
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features
     """

    # 신뢰도값 = training * alpha (최적값 40)
    confidence = sparse_data * alpha_val
    
    # 유저와 아이템의 사이즈
    user_size, item_size = sparse_data.shape
    
    # X(user matrix) , Y(item matrix) 랜덤값으로 초기화
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
    #Precompute I and lambda * I
    # I는 identifier matrix
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I

Still inside our implicit_als function we start the main iteration loop. Here we first precompute X-transpose-X and Y-transpose-Y as discussed earlier. We then have two inner loops where we first iterate over all users and update X and then do the same for all items and update Y.

In [75]:
""" Continuation of implicit_als function"""

# loop을 통해 유저와 아이템을 반복적으로 최적화
for i in xrange(iterations):
    print( 'iteration %d of %d' % (i+1, iterations))

    # Precompute Y-transpose-Y and X-transpose-X
    yTy = Y.T.dot(Y)
    xTx = X.T.dot(X)

    # 유저와 아이템을 loop을 통해 최적화
    for u in xrange(user_size):

        # 유저 값을 구함
        u_row = confidence[u,:].toarray() 

        # 선호도 p(u)값을 계산
        p_u = u_row.copy()
        p_u[p_u != 0] = 1.0

        
        # Calculate Cu and Cu - I
        
        CuI = sparse.diags(u_row, [0])
        Cu = CuI + Y_I

        # Put it all together and compute the final formula
        yT_CuI_y = Y.T.dot(CuI).dot(Y)
        yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
        X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)


    for i in xrange(item_size):

        # 아이템 컬럼을 구하고 전치
        i_row = confidence[:,i].T.toarray()

        # Calculate the binary preference p(i)
        p_i = i_row.copy()
        p_i[p_i != 0] = 1.0

        # Calculate Ci and Ci - I
        CiI = sparse.diags(i_row, [0])
        Ci = CiI + X_I

        # Put it all together and compute the final formula
        xT_CiI_x = X.T.dot(CiI).dot(X)
        xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
        Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

return X, Y

NameError: name 'xrange' is not defined